The first thing we have to do is checking if we have support for a GPU.

It's required to enable GPU hardware accelerator over `Edit > Notebook Settings`

In [1]:
!nvidia-smi

Tue Sep 13 12:22:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Next bunch of code is to set up `Stable Diffussion`

1. Install Dependencies

In [ ]:
!pip install diffusers==0.2.4
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 112 kB 28.2 MB/s 
     |████████████████████████████████| 120 kB 63.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 31.2 MB/s 
     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 6.6 MB 57.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 35.9 MB/s 


2. Enable external Colab widgets

This allows to connect to hugging face, which will be used to download stable difussion models

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

3. Connect to Hugging Face

For this step you will have to create an account for `Hugging Face` if you haven't already; and accept their terms and conditions. The widget have a link to the website where you would be doing that

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


4. Initialize `Stable Diffussion Pipeline` 

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True)

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.63k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/167M [00:00<?, ?B/s]

5. Move pipeline to GPU

In [ ]:
pipe = pipe.to("cuda")

6. Run!

In [ ]:
from torch import autocast
import io


def dream(prompt: str):
  with autocast("cuda"):
    image = pipe(prompt)["sample"][0]  

  byteIO = io.BytesIO()
  image.save(byteIO, format='PNG')
  return byteIO.getvalue()

----------------------------------

## CREATE THE API!!!

At this stage we have a system that can create images given a written prompt. Let's use **Fast API** to create an API which given a query `prompt="A pug dressed like Napoleon` generates the image and returns it back.

Example:

`http://your-public-address.com?prompt="A pug dressed like Napoleon"`

1. Install dependencies

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 745 kB 60.9 MB/s 
     |████████████████████████████████| 57 kB 6.0 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 80 kB 10.1 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=93b1598df51682b355725c0ab0a196f30b1143462678169425cdea7eda344a34
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


2. Create the API definition

In [ ]:
from fastapi import FastAPI, Response
import asyncio
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get("/")
async def root(prompt: str):
    return Response(content=dream(prompt), media_type="image/png")

3. Run the API

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [70]
INFO:uvicorn.error:Started server process [70]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://1afa-35-247-145-203.ngrok.io


0it [00:00, ?it/s]

INFO:     3.84.22.184:0 - "GET /?prompt= HTTP/1.1" 200 OK


0it [00:00, ?it/s]

...

INFO:     201.141.122.187:0 - "GET /?prompt=%22A%20chicken%20lost%20in%20time%20by%20Salvador%20Dali%22 HTTP/1.1" 200 OK


0it [00:00, ?it/s]

INFO:     201.141.122.187:0 - "GET /?prompt=%22A%20chicken%20lost%20in%20time%20by%20Salvador%20Dali%22 HTTP/1.1" 200 OK
INFO:     201.141.122.187:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


0it [00:00, ?it/s]

INFO:     201.141.122.187:0 - "GET /?prompt=chicken HTTP/1.1" 200 OK
